In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
#Kütüphanelerin eklenmesi
import numpy as np #Bu kütüphane lineer cebir için kullandığımız kütüphane fonksiyonlarını içeriyor
import pandas as pd # verilerimizi işlemek için pandas kütüphanasini kullanıyoruz(örn pd.read_scv)
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from utils import *
import json
import random
#from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish

from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import re
import string
from sklearn.metrics import classification_report, accuracy_score, make_scorer


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

# DEEP LEARNING IMPORTS
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix


from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from sklearn.pipeline import Pipeline


In [2]:
#column = ['tweets','duygu','preprocessing']
#df = pd.read_excel("../dataset/total.xlsx")

column = ['tweets','duygu']
df = pd.read_excel("../dataset/kemik_pos_neg.xlsx")


df.columns=column
#veri setinin gösterilmesi
df=df.drop_duplicates()
df['tweets']=df['tweets'].str.lower()


In [3]:
df.loc[df.duygu=="olumlu","duygu"]=1
df.loc[df.duygu=="olumsuz","duygu"]=0
df[:4]

In [4]:
df=df.sample(frac=1).reset_index(drop=True)
df

In [5]:
inputs=df['tweets'].to_numpy()
targets=df['duygu'].to_numpy()

In [6]:
def tf_idf (X_train,y_train,X_test,y_test):
    #Create a tokenizer, configured to only take into account the 20 most common words çok küçük olursa kelimeleri 
    #kaybederiz underfit yaparız
    tokenizer = Tokenizer(num_words=1000) #en yaygın kaç kelimeyi dikkate alacağı. Belirtilecek en iyi kelime sayısı #1000 yapan da var
    tokenizer.fit_on_texts(X_train) #keras tokenizer ile metni dictionary haline getiriyor.
    sequences_X_train = tokenizer.texts_to_sequences(X_train) #kelimelerin dictionarydeki karşılığı 
    #[[2, 1, 3], [2, 1], [4, 1], [5, 6]] şekline getiriliyor. 2-machine 1- learning 3-Knowledge 
    word_index = tokenizer.word_index #dictionarydeki kelimelerin sayısal karşılığı 'unk': 1, 'ürün': 2,
    max_length = 0
    for review_number in range(len(sequences_X_train)): #len(sequences_X_train) ile kaç tane [[2,3,4],[2,6]] var bulunuyor burda 2
        numberofwords=len(sequences_X_train[review_number]) #[2,3,4] içinde kaç tane şey var 3 burda
        if (numberofwords) > (max_length):
            max_length = numberofwords #tüm kelimelere bakıp en uzun olanı buluyor

    vocabulary=[]
    for key in word_index.keys():
        vocabulary.append(key)

    pipe = Pipeline([('count', CountVectorizer(vocabulary=vocabulary)),
                     ('tfid', TfidfTransformer())]).fit(X_train)

    res = dict(zip(vocabulary, pipe['tfid'].idf_))

    X_train = pad_sequences(sequences_X_train, maxlen=max_length) #ikili boyutlu matrise çevirip her cümelnin uzunluğunu eşit yapıyor.
    #En uzun cümle uzunluğuna tamamlanıyor.[[2 1 3] [0 2 1]] alt alta gelecek şekilde en uzun 6 ise 6x6 matris oluyor
    y_train = np.asarray(y_train) #tek boyutlu bir matris oluyor [1 1 0 ... 0 1 0] gibi

    sequences_X_test = tokenizer.texts_to_sequences(X_test) #train için yapılan gibi dictionary alınıyor
    X_test = pad_sequences(sequences_X_test, maxlen=max_length) #en uzun olana göre pad sequence yapılıyor
    y_test = np.asarray(y_test)


    unique_words = len(word_index) #word_index ile unique olan kelimeler alınıyor 0 dan başladığı için bir arttırılıyor
    total_words = unique_words + 1
    
    skipped_words = 0
    embedding_dim = 1 #embedding dim vector size ile aynı 
    embedding_vector=0
    embedding_matrix = np.zeros((total_words, embedding_dim))
    for word, index in tokenizer.word_index.items(): #kelime ve kelimenin dictionarydeki karşılığı alınıyor
        try:
            embedding_vector = res[word] #kelimenin word2vec karşılığı vektör olarak
        except:
            skipped_words = skipped_words+1
            pass
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector 
            
    embedding_layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

    return embedding_layer,X_train,y_train,X_test,y_test

In [7]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=10, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
acc,pre,rec,f1=[],[],[],[]
pat = 5
early_stopping = EarlyStopping(monitor='loss', patience=pat, verbose=True)
    
for train, test in kfold.split(inputs, targets):
    
    X_train=inputs[train].tolist()
    y_train=targets[train].tolist()
    X_test=inputs[test].tolist()
    y_test=targets[test].tolist()
    
    embedding_layer,X_train,y_train,X_test,y_test= tf_idf (X_train,y_train,X_test,y_test)
    
    modelCNN = Sequential() #cnn

    modelCNN.add(embedding_layer)
    modelCNN.add(Conv1D(filters=256, kernel_size=5, activation='relu')) #kernal size 5 yan yana beş kelimeye bakması
    modelCNN.add(MaxPooling1D(pool_size=2)) #tek satırlık 1d olduğu için
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(360, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(300, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(260, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(150, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(120, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Dense(80, activation='relu'))
    modelCNN.add(Dropout(0.3))
    modelCNN.add(Flatten()) #düzgünleştirmek için
    modelCNN.add(Dense(1, activation='sigmoid'))
    modelCNN.compile(loss='binary_crossentropy', optimizer='adam', 
                         metrics=['acc',tf.keras.metrics.Precision(),
                                  tf.keras.metrics.Recall()]) #binary cross çünkü sonucun pozitif yada negatif

    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    
    
    modelCNN.fit(X_train, y_train, epochs=50,callbacks=early_stopping)

    # evaluate
    loss, accuracy, precision, recall = modelCNN.evaluate(X_test, y_test)
    f1_score = 2 * (precision * recall) / (precision + recall)
    acc.append(accuracy)
    pre.append(precision)
    rec.append(recall)
    f1.append(f1_score)
    
    
    # Increase fold number
    fold_no = fold_no + 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 373, 1)            7044      
_________________________________________________________________
conv1d (Conv1D)              (None, 369, 256)          1536      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 184, 256)          0         
_________________________________________________________________
dropout (Dropout)            (None, 184, 256)          0         
_________________________________________________________________
dense (Dense)                (None, 184, 360)          92520     
_________________________________________________________________
dropout_1 (Dropout)          (None, 184, 360)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 184, 300)          1

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

Epoch 1/100
420/420 [==============================] - 132s 314ms/step - loss: 0.6727 - accuracy: 0.5895
Epoch 2/100
420/420 [==============================] - 133s 316ms/step - loss: 0.6749 - accuracy: 0.5874
Epoch 3/100
420/420 [==============================] - 161s 383ms/step - loss: 0.6652 - accuracy: 0.6064
Epoch 4/100
420/420 [==============================] - 231s 551ms/step - loss: 0.6653 - accuracy: 0.6043
Epoch 5/100
420/420 [==============================] - 248s 591ms/step - loss: 0.6620 - accuracy: 0.6075
Epoch 6/100
420/420 [==============================] - 218s 520ms/step - loss: 0.6595 - accuracy: 0.6082
Epoch 7/100
420/420 [==============================] - 197s 469ms/step - loss: 0.6585 - accuracy: 0.6115
Epoch 8/100
420/420 [==============================] - 196s 467ms/step - loss: 0.6575 - accuracy: 0.6172
Epoch 9/100
420/420 [==============================] - 208s 496ms/step - loss: 0.6561 - accuracy: 0.6150
Epoch 10/100
420/420 [==============================] -

In [11]:
print('test Accuracy: %f' % (Average(acc)))
print('test precision: %f' % (Average(pre)))
print('test recall: %f' % (Average(rec)))
print('test f1_score: %f' % (Average(f1)))

207/207 [==============================] - 8s 37ms/step - loss: 0.6630 - accuracy: 0.5748
test Accuracy: 57.478052
